In [1]:
import sys
sys.path.append('../') # add this project path to the system path to import own module

from Util.util import Constant, getMilliTime
from Util import util
import time
import requests
import pickle
from Ip.ipCheck import IpPool, Ip
from Account.accountCheck import AccountPool, Account
import threading
from hyper.contrib import HTTP20Adapter
import random

In [2]:
class MyThread(threading.Thread):
    def __init__(self, threadID, account, sleepSec, goods_id):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.account = account
        self.sleepSecRange = [sleepSec / 2, sleepSec]
        self.goods_id = goods_id
        self.buff_req_headers = { # request header for accessing market in buff
            ':authority': 'buff.163.com',
            ':scheme': 'https',
            'accept': 'application/json, text/javascript, */*; q=0.01',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'en-US,en;q=0.9',
            'sec-ch-ua': '"Chromium";v="112", "Google Chrome";v="112", "Not:A-Brand";v="99"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"Linux"',
            'sec-fetch-dest': 'empty',
            'sec-fetch-mod': 'cors',
            'sec-fetch-site': 'same-origin',
            'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
            'x-requested-with': 'XMLHttpRequest'
        } 
        self.sessions = requests.session() # the session for accessing buff
        self.sessions.mount(Constant.domain, HTTP20Adapter())
        self.sessions.headers = self.buff_req_headers
    
    def notification(self, session, domain, cookie, csrf_token, proxies):
        _path = Constant.notification_without_timestamp_addr + getMilliTime()
        _url = Constant.domain + _path
        _header = {
            ':method': 'GET',
            ':path': _path,
            'cookie': cookie + csrf_token,
            'referer': domain + '/?game=csgo'
        }
        return session.get(_url, headers = _header, proxies = proxies)

    def get_goods_data(self, session, domain, cookie, csrf_token, goods_id, proxies):
        # 裂空武器箱:781534
        _path = Constant.getGoodsAddr(goods_id, getMilliTime())
        _url = Constant.domain + _path
        _header = {
            ':method': 'GET',
            ':path': _path,
            'cookie': cookie + 'csrf_token=' + csrf_token,
            'referer': domain + '/goods/' + goods_id,
        }
        return session.get(_url, headers = _header, proxies = proxies)
        
    def run(self):
        print ("Starting thread: " + self.threadID)
        
        while(True):
            time.sleep(random.randint(self.sleepSecRange[0], self.sleepSecRange[1]))
            resp = self.get_goods_data(self.sessions, Constant.domain, self.account.cookie, self.account.csrf_token, self.goods_id, self.account.ipObj.proxies)
            item = resp.json()['data']['items'][0]
            # print('get_goods_data: \n')
            # print(item)
            self.account.csrf_token = resp.headers.get(b'set-cookie').decode("utf-8").split(';')[0].split('=')[1]

class Main(object):
    def __init__(self):
        self.account_list = self.getAccountPool()
        self.account_len = len(self.account_list)
        self.run()
        
    def run(self):
        print('accounts number: ' + str(len(self.account_list)))
        for i in range(0, self.account_len):
            time.sleep(1)
            MyThread('Thread ' + str(i), self.account_list[i], self.account_len, '900464').start()
    
    def getAccountPool(self):
        with open(Constant.account_pool_dir, 'rb') as fin: 
            return pickle.load(fin)

In [3]:
Main()

accounts number: 2
Starting thread: Thread 0
Starting thread: Thread 1
